# Image Classification with fastai

In [ ]:
from fastai.vision import *
import fastai

In [ ]:
# We can verify that we don’t have any corrupt images using the verify_images method.
import os
path = '../input/animals-jpg'
for folder in ('cat', 'buterfly','hors','pandas','parots'):
    print(folder)
    verify_images(os.path.join(path, folder), delete=True, max_size=500)

In [ ]:

np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=path, valid_pct=0.2,
                                  ds_tfms=get_transforms(), size=224, num_workers=0,bs=16).normalize(imagenet_stats)
# Setting num_workers=0 prevents crushing!

In [ ]:
data.classes

In [ ]:
data.show_batch(rows=3, figsize=(7, 8))

In [ ]:
from fastai.metrics import error_rate # 1 - accuracy
learn = create_cnn(data, models.resnet34, metrics=error_rate)
# The created model uses the resnet34 architecture, with weights pretrained on the imagenet dataset.
# By default, only the fully connected layers at the top are unfrozen (can be trained), 
# which if you are familiar with transfer learning makes perfect sense.

In [ ]:
# defaults.device = torch.device('cuda') # makes sure the gpu is used
learn.fit_one_cycle(3)

In [ ]:
# Will take too long without GPU
learn.unfreeze() # must be done before calling lr_find
learn.lr_find()
learn.recorder.plot()

In [ ]:
# Might take too long without GPU
learn.fit_one_cycle(4, max_lr=slice(3e-5, 3e-4))

In [ ]:
learn.save('animal-detection-stage-1')

## Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(9, figsize=(15,15))
# Show images in `top_losses` along with their prediction, actual, loss, and probability of actual class.

In [ ]:
learn.summary()

# Excercise 5

Create clasification of 5 different classes (animals or something else). Optionally execute on Kaggle notebbok and send and share it with a group.